<a href="https://colab.research.google.com/github/deanhoperobertson/Named-Enitty-Recognition/blob/master/BiGRU-CRF%20Model%20(%2BUncased%20Embeddings).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Embeddings Bi-LSTM-CRF Model

In [1]:
from google.colab import files
uploaded = files.upload()

Saving prepro.py to prepro.py


In [2]:
!sudo pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-7v6o6ji0
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-7v6o6ji0
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101066 sha256=280bc01968d84ff3f9793f0dff41a05a90c52c422a583ffee874dc544f24f01a
  Stored in directory: /tmp/pip-ephem-wheel-cache-sf02t06t/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [3]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 757kB 45.0MB/s 


In [0]:
import pandas as pd
import numpy as np
import urllib.request
from sklearn.model_selection import train_test_split

#cusotm packages
from prepro import readfile, readstring, partial_tags

#keras and tensorflow packages
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, GRU
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy


#evaluation
from sklearn_crfsuite.metrics import flat_classification_report,flat_f1_score,flat_precision_score

In [0]:
#import data from my github repo
train_url = "https://raw.githubusercontent.com/deanhoperobertson/Masters-/master/Thesis/Code/Data/train.txt"
test_url = "https://raw.githubusercontent.com/deanhoperobertson/Masters-/master/Thesis/Code/Data/test.txt"
train = urllib.request.urlopen(train_url).read()
test = urllib.request.urlopen(test_url).read()
train = train.decode('utf-8')
test = test.decode('utf-8')

#preproces the txt file
train = readstring(train)
test = readstring(test)

#create corpus
corpus = train.copy()
corpus.extend(test)

In [6]:
words = []
tags = []
for sentence in corpus:
    for word in sentence:
        words.append(word[0])
        tags.append(word[1])
        
tags = partial_tags(tags)

setlabel=list(set(tags))
setlabel.remove('O')
        
words= [w.lower() for w in words]

n_words = len(set(words))
print("Number of words in the dataset: ", n_words)
n_tags = len(list(set(tags)))
print("Number of Labels: ", n_tags)

Number of words in the dataset:  24277
Number of Labels:  5


In [7]:
# The first entry is reserved for PAD
tag2idx = {t: i+1 for i, t in enumerate(list(set(tags)))}
tag2idx["PAD"] = 0
# Vocabulary Key:tag_index -> Value:Label/Tag
idx2tag = {i: w for w, i in tag2idx.items()}
print("The labels LOC (location) is identified by the index: {}".format(tag2idx["ORG"]))

The labels LOC (location) is identified by the index: 3


In [8]:
word2idx = {w: i + 2 for i, w in enumerate(list(set(words)))}
word2idx["UNK"] = 1 # Unknown words
word2idx["PAD"] = 0 # Padding
# Vocabulary Key:token_index -> Value:word
idx2word = {i: w for w, i in word2idx.items()}
print("The word 'rejects' is identified by the index: {}".format(word2idx["rejects"]))

The word 'rejects' is identified by the index: 9169


In [9]:
#Find the maxium length of the all the sentences in the corpus
length = []
for sentence in corpus:
  length.append(len(sentence))

MAX_LEN= max(length)
print("The maxium length of sentence is:",max(length))

The maxium length of sentence is: 124


In [10]:
# Convert each sentence from list of Token to list of word_index
X = [[word2idx[w[0].lower()] for w in s] for s in train]

# Padding each sentence to have the same lenght
X = pad_sequences(maxlen=MAX_LEN, sequences=X, padding="post", value=word2idx["PAD"])
X[0]

array([ 6999,  9169, 10255, 11956, 15456,  3807, 19558, 22108,   305,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0], dtype=int32)

In [11]:
def get_labeler(word):
  if word =="O":
    return word
  elif "MISC" in word:
    return word[-4:]
  else:
    return word[-3:]

# Convert Tag/Label to tag_index
y = [[tag2idx[get_labeler((w[1]))] for w in s] for s in train]

# Padding each sentence to have the same lenght
y = pad_sequences(maxlen=MAX_LEN, sequences=y, padding="post", value=tag2idx["PAD"])
y[0]

array([3, 5, 1, 5, 5, 5, 1, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [0]:
# One-Hot encode categorical labels
y = [to_categorical(i, num_classes=n_tags+1) for i in y]

## Adding Embeddings

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [14]:

#root_path = "/content/drive/My Drive/glove.6B.50d.txt"
#root_path = "/content/drive/My Drive/glove.6B.100d.txt"
#root_path = "/content/drive/My Drive/glove.6B.200d.txt"
root_path = "/content/drive/My Drive/glove.6B.300d.txt"
#root_path = "/content/drive/My Drive/wiki-news-300d-1M.vec"

embeddings_index={}
f = open(root_path, encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [0]:
#create emedding matrix

EMBEDDING=300
embedding_matrix = np.zeros((len(word2idx) + 1, EMBEDDING))
for word, i in word2idx.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [18]:
# Model definition

input = Input(shape=(MAX_LEN,))


emb = Embedding(len(word2idx)+1,output_dim=EMBEDDING,
                  weights=[embedding_matrix],input_length=MAX_LEN,
                  trainable=False,mask_zero=True)(input)

lstm1 = Bidirectional(GRU(units=64, return_sequences=True,
                           recurrent_dropout=0.1))(emb)

lstm2 = Bidirectional(GRU(units=64, return_sequences=True,
                           recurrent_dropout=0.1))(lstm1)

model_2 = TimeDistributed(Dense(10, activation="relu"))(lstm2)
crf = CRF(n_tags+1)  # CRF layer
out = crf(model_2)  # output
model_2 = Model(input, out)

W0805 19:38:16.583789 140248680134528 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0805 19:38:16.720328 140248680134528 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:2974: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [20]:
model_2.compile(optimizer='adam', loss=crf_loss,metrics=[crf_viterbi_accuracy])
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 124)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 124, 300)          7284000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 124, 128)          140160    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 124, 128)          74112     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 124, 10)           1290      
_________________________________________________________________
crf_1 (CRF)                  (None, 124, 6)            114       
Total params: 7,499,676
Trainable params: 215,676
Non-trainable params: 7,284,000
____________________________________________________________

In [21]:
%%time
BATCH_SIZE = 200
EPOCHS=16

history = model_2.fit(X, np.array(y), batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.2, verbose=1)

Train on 11232 samples, validate on 2809 samples
Epoch 1/16
11232/11232 [==============================] - 60s 5ms/step - loss: 33.4665 - crf_viterbi_accuracy: 0.7502 - val_loss: 29.0574 - val_crf_viterbi_accuracy: 0.8579
Epoch 2/16
11232/11232 [==============================] - 56s 5ms/step - loss: 32.9226 - crf_viterbi_accuracy: 0.8873 - val_loss: 28.8595 - val_crf_viterbi_accuracy: 0.9227
Epoch 3/16
11232/11232 [==============================] - 56s 5ms/step - loss: 32.7721 - crf_viterbi_accuracy: 0.9381 - val_loss: 28.7976 - val_crf_viterbi_accuracy: 0.9387
Epoch 4/16
11232/11232 [==============================] - 56s 5ms/step - loss: 32.7216 - crf_viterbi_accuracy: 0.9515 - val_loss: 28.7633 - val_crf_viterbi_accuracy: 0.9478
Epoch 5/16
11232/11232 [==============================] - 56s 5ms/step - loss: 32.6901 - crf_viterbi_accuracy: 0.9605 - val_loss: 28.7400 - val_crf_viterbi_accuracy: 0.9539
Epoch 6/16
11232/11232 [==============================] - 55s 5ms/step - loss: 32.6690

## Predict on Training Set

In [22]:
pred = model_2.predict(X, verbose=1)

14041/14041 [==============================] - 156s 11ms/step


In [23]:
# TRain Eval
preds = np.argmax(pred, axis=-1)
y_tr_true = np.argmax(y, -1)

# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in preds]
y_tr_true_tag = [[idx2tag[i] for i in row] for row in y_tr_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag)
print(report)

score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted')
print(score)

              precision    recall  f1-score   support

         LOC       0.97      0.94      0.95      8297
        MISC       0.94      0.87      0.90      4593
           O       0.99      0.99      0.99    169578
         ORG       0.87      0.94      0.90     10025
         PAD       1.00      1.00      1.00   1537463
         PER       0.98      0.98      0.98     11128

    accuracy                           1.00   1741084
   macro avg       0.96      0.95      0.96   1741084
weighted avg       1.00      1.00      1.00   1741084

0.9982427702486313


In [24]:
report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag,labels=setlabel)
print(report)
#F1 Score
score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted',labels=setlabel)
print(score)

              precision    recall  f1-score   support

        MISC       0.94      0.87      0.90      4593
         PER       0.98      0.98      0.98     11128
         ORG       0.87      0.94      0.90     10025
         LOC       0.97      0.94      0.95      8297

   micro avg       0.94      0.94      0.94     34043
   macro avg       0.94      0.93      0.94     34043
weighted avg       0.94      0.94      0.94     34043

0.9420775747587912


## Predict on Test Set

In [0]:
# Convert each sentence from list of Token to list of word_index
X_test = [[word2idx[w[0].lower()] for w in s] for s in test]

# Padding each sentence to have the same lenght
X_test = pad_sequences(maxlen=MAX_LEN, sequences=X_test, padding="post", value=word2idx["PAD"])
X_test

# Convert Tag/Label to tag_index
y_test = [[tag2idx[get_labeler((w[1]))] for w in s] for s in test]

# Padding each sentence to have the same lenght
y_test = pad_sequences(maxlen=MAX_LEN, sequences=y_test, padding="post", value=tag2idx["PAD"])

# One-Hot encode categorical labels
y_test = [to_categorical(i, num_classes=n_tags+1) for i in y_test]

In [26]:
pred = model_2.predict(X_test, verbose=1)

3453/3453 [==============================] - 38s 11ms/step


In [27]:
#pred_cat = model.predict(X_tr)
preds = np.argmax(pred, axis=-1)
y_tr_true = np.argmax(y_test, -1)

# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in preds]
y_tr_true_tag = [[idx2tag[i] for i in row] for row in y_tr_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag,labels=setlabel)
print(report)

score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted',labels=setlabel)
print(score)


              precision    recall  f1-score   support

        MISC       0.72      0.70      0.71       918
         PER       0.96      0.93      0.95      2773
         ORG       0.76      0.82      0.79      2496
         LOC       0.88      0.86      0.87      1925

   micro avg       0.85      0.86      0.85      8112
   macro avg       0.83      0.83      0.83      8112
weighted avg       0.85      0.86      0.85      8112

0.8542370714184102
